# LightGBMの推論用カスタムコンテナを構築し、SageMakerによる推論の仕組みを深く理解する

## アジェンダ
* パターン3のカスタムコンテナを作成
    * 学習編で実施済み
* パターン2のカスタムコンテナを作成：コンテナファイルを外部から指定する
    * Inference-Toolkitと、MMSをインストールしたコンテナを作成
* XGBoostビルトインコンテナで、LightBGMの推論を実施する
    * requirements.txtを用意する
    * inference.pyをお作用に従って作る

# Lab : LightGBMのカスタムコンテナを通して、SageMakerの動作を理解する

LightGBMがインストールされたカスタムコンテナを構築し、SageMaker Trainingジョブで学習を行います。
カスタムコンテナの挙動を観察し、SageMakerの動作について理解を深めます。

ノートブックは20分程度で実行できます。

# 0.実行環境確認
本ノートブックは、SageMakerノートブックインスタンス上で動作確認しています。
* インスタンスタイプ：ml.t3.medium
* カーネル：conda_python3

## 0-1.pythonバージョン確認

In [1]:
#Pythonのバージョン情報
import sys
sys.version # 3.8.12

'3.8.12 | packaged by conda-forge | (default, Oct 12 2021, 21:59:51) \n[GCC 9.4.0]'

In [2]:
# Pythonのバージョン確認 (システムコマンド使用）
!python -V # 3.8.12

Python 3.8.12


## 0-2.SageMakerSDKバージョン確認

Amazon SageMaker Python SDKは、Amazon SageMaker上で機械学習されたモデルをトレーニングおよびデプロイするためのオープンソースライブラリです。

このSDKを使用すると、一般的な深層学習フレームワーク、Amazonが提供するアルゴリズム、またはSageMaker互換のDockerイメージに組み込まれた独自のアルゴリズムを使ってモデルをトレーニングおよびデプロイすることができます。

* ドキュメント : https://sagemaker.readthedocs.io/en/stable/
* GitHub : https://github.com/aws/sagemaker-python-sdk

SageMakerSDK をインポートすると、バケットが作成されます。  
sagemaker-＜region＞-＜account＞

In [3]:
# SageMakerSDK のバージョン確認
import sagemaker
print('Current SageMaker Python SDK Version ={0}'.format(sagemaker.__version__)) # 2.110.0

Current SageMaker Python SDK Version =2.101.1


# 1.データ準備

学習、推論で利用するデータを準備します。

scikit-learn付属の、ボストン住宅価格データセットを利用します。(注：バージョン1.2から除外されます）  
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html

以下のスクリプトを参考にしています。

https://github.com/aws-samples/amazon-sagemaker-local-mode/blob/main/lightgbm_bring_your_own_container_local_training_and_serving/lightgbm_bring_your_own_container_local_training_and_serving.py

In [4]:
import sklearn
sklearn.__version__ # 1.0.1

'1.0.1'

In [5]:
import pandas as pd
pd.__version__ # 1.3.4

'1.3.4'

## 1-1. データロード

In [6]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

In [7]:
data = load_boston() # 1.2でデータセットがなくすという警告が出ますが動作に影響ありません

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the Calif

## 1-2. 特徴量生成（Feature Engineering）
本ノートブックでは実施しません。そのままデータを利用します。

## 1-3. データ分割
学習用（train）、評価用（validation）、テスト用（test）にデータを分割します。  
train:val:test = 3(60%):1(20%):1(20%)に分割します。  

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=45)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=45)

trainX = pd.DataFrame(X_train, columns=data.feature_names)
trainX['target'] = y_train

valX = pd.DataFrame(X_val, columns=data.feature_names)
valX['target'] = y_val

testX = pd.DataFrame(X_test, columns=data.feature_names)

In [9]:
# 確認
trainX.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9
1,0.33983,22.0,5.86,0.0,0.431,6.108,34.9,8.0555,7.0,330.0,19.1,390.18,9.16,24.3
2,0.10469,40.0,6.41,1.0,0.447,7.267,49.0,4.7872,4.0,254.0,17.6,389.25,6.05,33.2
3,6.80117,0.0,18.10,0.0,0.713,6.081,84.4,2.7175,24.0,666.0,20.2,396.90,14.70,20.0
4,1.35472,0.0,8.14,0.0,0.538,6.072,100.0,4.1750,4.0,307.0,21.0,376.73,13.04,14.5


In [10]:
# 確認
valX.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.03150,95.0,1.47,0.0,0.403,6.975,15.3,7.6534,3.0,402.0,17.0,396.90,4.56,34.9
1,0.51183,0.0,6.20,0.0,0.507,7.358,71.6,4.1480,8.0,307.0,17.4,390.07,4.73,31.5
2,19.60910,0.0,18.10,0.0,0.671,7.313,97.9,1.3163,24.0,666.0,20.2,396.90,13.44,15.0
3,0.95577,0.0,8.14,0.0,0.538,6.047,88.8,4.4534,4.0,307.0,21.0,306.38,17.28,14.8
4,0.09604,40.0,6.41,0.0,0.447,6.854,42.8,4.2673,4.0,254.0,17.6,396.90,2.98,32.0


In [11]:
# 確認
testX.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.25387,0.0,6.91,0.0,0.4480,5.399,95.3,5.8700,3.0,233.0,17.9,396.90,30.81
1,0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05
2,4.64689,0.0,18.10,0.0,0.6140,6.980,67.6,2.5329,24.0,666.0,20.2,374.68,11.66
3,3.67367,0.0,18.10,0.0,0.5830,6.312,51.9,3.9917,24.0,666.0,20.2,388.62,10.58
4,0.29819,0.0,6.20,0.0,0.5040,7.686,17.0,3.3751,8.0,307.0,17.4,377.51,3.92


In [12]:
# 確認
y_test[0:5]

array([14.4, 33. , 29.8, 21.2, 46.7])

## 1-4.データ保存
ローカル、S3それぞれにデータを保存します。

### 1-4-1.ローカルへ保存

In [13]:
# ディレクトリ作成
from pathlib import Path

Path('./data/train').mkdir(parents=True, exist_ok=True)
Path('./data/valid').mkdir(parents=True, exist_ok=True)
Path('./data/test').mkdir(parents=True, exist_ok=True)

In [14]:
# ローカルへ保存
local_train = './data/train/boston_train.csv'
local_valid = './data/valid/boston_valid.csv'
local_test = './data/test/boston_test.csv'

trainX.to_csv(local_train, header=None, index=False)
valX.to_csv(local_valid, header=None, index=False)
testX.to_csv(local_test, header=None, index=False)

### 1-4-2.S3へ保存

一意のバケット作成のために、sgemaker.Session().default_bucket()を利用します。

https://sagemaker.readthedocs.io/en/stable/api/utility/session.html#sagemaker.session.Session

sagemaker-＜region＞-＜accoutid＞　を取得することができます。

In [15]:
bucket_name = sagemaker.Session().default_bucket()
region_name = sagemaker.Session().boto_region_name
account_id =  sagemaker.Session().account_id()

In [16]:
# 確認
print(bucket_name)
print(region_name)
print(account_id)

sagemaker-ap-northeast-1-805433377179
ap-northeast-1
805433377179


In [17]:
# バケット作成(SageMakerSDKのインポート時作成されています。他のバケット作成時に利用ください)
#import boto3

#s3_resource = boto3.resource('s3')
#s3_resource.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region_name})

In [18]:
# S3へ保存
train_s3 = sagemaker.s3.S3Uploader.upload('./data/train/boston_train.csv', f's3://{bucket_name}/demo_lightgbm/train')
valid_s3 = sagemaker.s3.S3Uploader.upload('./data/valid/boston_valid.csv', f's3://{bucket_name}/demo_lightgbm/valid')

# 2.LightGBMカスタムコンテナの構築


カスタムコンテナの作成には大きく分けて3つのパターンがあります。詳細は以下のブログを参考ください。

https://aws.amazon.com/jp/blogs/news/sagemaker-custom-containers-pattern-training/

SageMakerの動作を理解するためにパターン3のベースイメージ + カスタムレイヤー方式を採用します。

## 2-1. Dockerfileの確認

資材はこちらのノートブックを参考に準備しています。

https://github.com/aws-samples/amazon-sagemaker-local-mode/tree/main/lightgbm_bring_your_own_container_local_training_and_serving/container

In [ ]:
!pygmentize ./container/Dockerfile

## 2-2. dockerイメージの build & push

ビルド&pushには7分ほどかかります。

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-lightgbm-regression

cd container

chmod +x lightgbm_regression/train
chmod +x lightgbm_regression/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to ap-northeast-1 if none defined)
region=$(aws configure get region)
region=${region:-ap-northeast-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

## 2-3. 学習前設定
ECRでpushしたコンテナのURIを確認

AWSコンソールでECRに移動し、作成したコンテナがあることを確認します。

image URIを取得し、以下にはりつけます。

In [19]:
# 確認
print(bucket_name)
print(region_name)
print(account_id)

sagemaker-ap-northeast-1-805433377179
ap-northeast-1
805433377179


In [20]:
image_uri = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/sagemaker-lightgbm-regression'

In [21]:
# 確認
image_uri

'805433377179.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-lightgbm-regression'

In [22]:
hyperparameters={'boosting_type': 'gbdt',
            'objective': 'regression',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0}

## 2-4.ローカル学習の実行
ECRからビルドしたイメージを持ってきて、ローカルのdockerでビルドして、実行する

In [ ]:
# ローカルファイルのパスを設定（S3パス指定も可）
train_location = 'file://'+local_train
valid_location = 'file://'+local_valid

print(train_location)
print(valid_location)

In [27]:
from sagemaker.estimator import Estimator

In [28]:
from sagemaker import get_execution_role

role = get_execution_role()

In [29]:
# 確認
role

'arn:aws:iam::805433377179:role/service-role/AmazonSageMaker-ExecutionRole-20220807T102095'

SageMakerのEstimatorを作成します。

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html

In [30]:
local_lightgbm = Estimator(
    image_uri,
    role,
    instance_count=1,
    instance_type="local",
    hyperparameters=hyperparameters
    )

fitメソッドで学習ジョブを発行します

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.fit

In [ ]:
local_lightgbm.fit({'train':train_location, 'validation': valid_location})

ローカルモードの学習結果は

Amazon S3
Buckets
sagemaker-us-west-2-805433377179
sagemaker-lightgbm-regression-2022-10-03-06-17-32-054/

に出力されます。


## 2-5.ローカルデプロイ

serializer : インプットデータの形式を指定します。
https://sagemaker.readthedocs.io/en/stable/v2.html

In [ ]:
local_predictor = local_lightgbm.deploy(1, 'local', serializer=sagemaker.serializers.CSVSerializer()) 

In [ ]:
!docker ps

In [ ]:
# 起動中のコンテナを停止する場合
#!docker stop XXXXXXXXXXX #XXXXXXXXXXXは CONtAINER ID
#!docker ps

## 2-6.ローカルエンドポイントで推論実施

In [ ]:
# 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = local_predictor.predict(payload).decode('utf-8')
print('=' * 20)
print(predicted)

## 2-7.学習ジョブを発行
次は、ローカルモードではなく、
同じカスタムコンテナで、学習ジョブを実行します。

In [ ]:
# 確認
print(train_s3)
print(valid_s3)

In [ ]:
est_lightgbm = Estimator(
    image_uri,
    role,
    instance_count=1,
    instance_type="ml.m4.2xlarge", # インスタンスタイプを指定
    hyperparameters=hyperparameters)

In [ ]:
est_lightgbm.fit({'train':train_s3, 'validation': valid_s3})

学習には3分ほど時間がかかります。

課金されるのは75秒ほどです。

## 2-8.エンドポイントにデプロイ

デプロイすると、
SageMaker は docker run <image> serveを実行します。

    
デプロイには3分ほどかかります。

In [23]:
from sagemaker.predictor import csv_serializer

deployメソッドで、推論エンドポイントをデプロイします。

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.deploy

In [ ]:
predictor = est_lightgbm.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer, wait=True)

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor.predict(payload).decode('utf-8')
print(predicted)

# 3. 実行ファイルを外部から指定する

「2.LightGBMカスタムコンテナの構築」ではカスタムコンテナ内に学習起動スクリプトtrainを配置しましたが、
ソースコードを修正するごとにコンテナを作り替える必要があります。

保守性を上げるには、コンテナ（環境）とソースコードを分けた方がいい場合もあります。
以下では外部からスクリプトファイルを指定する方法を紹介します。

## 3-0.SageMaker Training Toolkitとは
外部からスクリプトを指定するためには、SageMaker Training Toolkitを導入します。

https://github.com/aws/sagemaker-training-toolkit


trainコマンドが  
/opt/conca/bin/train  
にインストールされます。  


先程のdockerfileに追記します。
資材からは、trainを除外しておきます。trainを含んだままの場合、
docker run <image> train
を実行したときに、カレントディレクトリのtrainスクリプトが実行されてしまい、training toolkitが導入した　trainコマンドが実行できないためです。

## 3-1. Dockerfile確認

In [ ]:
!pygmentize ./container_smtrtoolkit/Dockerfile

## 3-2. build & push

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-toolkit

#cd container
cd container_smtrtoolkit ### 変更点

#chmod +x lightgbm_regression/train ### 変更点
chmod +x lightgbm_regression/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to ap-northeast-1 if none defined)
region=$(aws configure get region)
region=${region:-ap-northeast-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}


## 3-3.学習(ローカル)

In [ ]:
image_uri_toolkit = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/sagemaker-toolkit'

In [ ]:
# 確認
image_uri_toolkit

In [ ]:
est_lightgbm_toolkit = Estimator(
    image_uri_toolkit,
    role,
    instance_count=1,
    instance_type="local",
    hyperparameters=hyperparameters,
    entry_point='./src/train_practice.py'
    )

In [ ]:
est_lightgbm_toolkit.fit({'train':train_s3, 'validation': valid_s3})

ローカルモードで学習することができました。別のスクリプトを指定してみましょう。

In [ ]:
est_lightgbm_toolkit2 = Estimator(
    image_uri_toolkit,
    role,
    instance_count=1,
    instance_type="local",
    hyperparameters=hyperparameters,
    #entry_point='./src/train_practice.py'
    entry_point='./src/train_practice.sh' ### シェルスクリプトに変更
    )

In [ ]:
est_lightgbm_toolkit2.fit({'train':train_s3, 'validation': valid_s3})

コンテナ外部から任意のファイルを実行することが確認できました。

## （optional）4. カスタムコンテナを使わず、built-inコンテナのrequirement.txtにlightgbmを記載して実行する



過去バージョン（1.3-3, 1.2-2, 1.2-1, 1.0-1)はこちら

https://github.com/aws/sagemaker-xgboost-container/releases


In [24]:
container_uri = sagemaker.image_uris.retrieve("xgboost", region_name, "1.5-1")

In [25]:
container_uri

'354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-xgboost:1.5-1'

In [67]:
est_xgb = Estimator(
    container_uri, # xgboostのbuilt-inコンテナ
    role,
    instance_count=1,
    instance_type="local",
    hyperparameters=hyperparameters,
    source_dir='./src_builtin_container',
    entry_point='train_practice.py'
    )

In [68]:
[1,2,3,4,5]

[1, 2, 3, 4, 5]

In [69]:
est_xgb.fit({'train':train_s3, 'validation': valid_s3})

Creating 46cx9f4qks-algo-1-cn0lk ... 
Creating 46cx9f4qks-algo-1-cn0lk ... done
Attaching to 46cx9f4qks-algo-1-cn0lk
46cx9f4qks-algo-1-cn0lk | [2022-10-15 06:58:32.707 ee3916c08eca:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
46cx9f4qks-algo-1-cn0lk | [2022-10-15:06:58:32:INFO] Imported framework sagemaker_xgboost_container.training
46cx9f4qks-algo-1-cn0lk | [2022-10-15:06:58:32:INFO] Failed to parse hyperparameter boosting_type value gbdt to Json.
46cx9f4qks-algo-1-cn0lk | Returning the value itself
46cx9f4qks-algo-1-cn0lk | [2022-10-15:06:58:32:INFO] Failed to parse hyperparameter objective value regression to Json.
46cx9f4qks-algo-1-cn0lk | Returning the value itself
46cx9f4qks-algo-1-cn0lk | [2022-10-15:06:58:32:INFO] No GPUs detected (normal if no gpus installed)
46cx9f4qks-algo-1-cn0lk | [2022-10-15:06:58:32:INFO] Invoking user training script.
46cx9f4qks-algo-1-cn0lk | [2022-10-15:06:58:33:INFO] Module train_practice does not provide a setup.py. 
46cx9f4qks-algo-1-cn0

## 4-2. 推論実施

### 4-2-1.デプロイ

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.deploy


デプロイの際に、ソースコードを指定するにはどうしたらいいのか？

https://www.youtube.com/watch?v=sngNd79GpmE&t=596s


ポイント：あらためて、Estimatorを定義する必要がある。

In [71]:
# 確認
print(est_xgb)
print(est_xgb.base_job_name)
print(est_xgb.checkpoint_local_path)
print(est_xgb.checkpoint_s3_uri)
print(est_xgb.code_channel_name)
print(est_xgb.code_location)
print(est_xgb.code_uri)
print(est_xgb.collection_configs)
print(est_xgb.CONTAINER_CODE_CHANNEL_SOURCEDIR_PATH)
print(est_xgb.container_log_level)
print(est_xgb.debugger_hook_config)
print(est_xgb.debugger_rule_configs)
print(est_xgb.debugger_rules)
print(est_xgb.dependencies)
print(est_xgb.deploy_instance_type)
print(est_xgb.disable_profiler)
print(est_xgb.enable_sagemaker_metrics)
print(est_xgb.encrypt_inter_container_traffic)
print(est_xgb.entry_point)
print(est_xgb.environment)
print(est_xgb.git_config)
print(est_xgb.image_uri)
print(est_xgb.input_mode)
print(est_xgb.instance_count)
print(est_xgb.model_uri)
print(est_xgb.model_data)


sagemaker-xgboost
None
None
code
None
None
set()
/opt/ml/input/data/code/sourcedir.tar.gz
20
None
[]
[]
[]
None
False
None
False
train_practice.py
None
None
354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-xgboost:1.5-1
File
1
None
s3://sagemaker-ap-northeast-1-805433377179/sagemaker-xgboost-2022-10-15-06-58-29-510/model.tar.gz


### serve用のファイルは、.py かつ、作法に従う必要がある。

MMSは.pyを扱うように設計されているため。

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Estimator

In [ ]:
# デプロイ
predictor_xgb = est_xgb.deploy(1, 'local', serializer=csv_serializer, wait=True)

Attaching to pz2ljof9hn-algo-1-5ftd5
pz2ljof9hn-algo-1-5ftd5 | [2022-10-15:06:59:32:INFO] No GPUs detected (normal if no gpus installed)
pz2ljof9hn-algo-1-5ftd5 | [2022-10-15:06:59:32:INFO] No GPUs detected (normal if no gpus installed)
pz2ljof9hn-algo-1-5ftd5 | [2022-10-15:06:59:32:INFO] nginx config: 
pz2ljof9hn-algo-1-5ftd5 | worker_processes auto;
pz2ljof9hn-algo-1-5ftd5 | daemon off;
pz2ljof9hn-algo-1-5ftd5 | pid /tmp/nginx.pid;
pz2ljof9hn-algo-1-5ftd5 | error_log  /dev/stderr;
pz2ljof9hn-algo-1-5ftd5 | 
pz2ljof9hn-algo-1-5ftd5 | worker_rlimit_nofile 4096;
pz2ljof9hn-algo-1-5ftd5 | 
pz2ljof9hn-algo-1-5ftd5 | events {
pz2ljof9hn-algo-1-5ftd5 |   worker_connections 2048;
pz2ljof9hn-algo-1-5ftd5 | }
pz2ljof9hn-algo-1-5ftd5 | 
pz2ljof9hn-algo-1-5ftd5 | http {
pz2ljof9hn-algo-1-5ftd5 |   include /etc/nginx/mime.types;
pz2ljof9hn-algo-1-5ftd5 |   default_type application/octet-stream;
pz2ljof9hn-algo-1-5ftd5 |   access_log /dev/stdout combined;
pz2ljof9hn-algo-1-5ftd5 | 
pz2ljof9hn-algo

## エラー

RuntimeError: Model /opt/ml/model/model.tar.gz cannot be loaded:

In [59]:
# 改めて、Estimatorを定義する
est_xgb_serve = Estimator(
    container_uri, # xgboostのbuilt-inコンテナ
    role,
    model_data=est_xgb.model_data, ### 学習で保存されたモデルのURI
    instance_count=1,
    instance_type="local",
    #hyperparameters=hyperparameters,
    source_dir='./src_builtin_container_serve',
    entry_point='serve.py' 
    )

In [60]:
# デプロイ
predictor_xgb = est_xgb_serve.deploy(1, 'local', serializer=csv_serializer, wait=True)

ValueError: Estimator is not associated with a training job

ValueError: Estimator is not associated with a training job

# エラー　trainingjobとの紐付け

https://stackoverflow.com/questions/63340328/how-to-define-a-sagemaker-estimator-object-using-a-pre-trained-model-and-then-de

In [ ]:
est_xgb_serve2 = est_xgb_serve.attach(TrainingJobName)

In [57]:
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


In [58]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor_xgb.predict(payload).decode('utf-8')
print(predicted)

NameError: name 'predictor_xgb' is not defined

# END of Containts ===============

# 5.後片付け
予期せぬ課金を防ぐために、以下のリソースを削除します。

* SageMaker 推論エンドポイント
* ECR
* S3
* SageMakerノートブックインスタンス

# 参考
* SageMaker のtrainingジョブを理解する
    * https://github.com/aws-samples/aws-ml-jp/tree/main/sagemaker/sagemaker-traning/tutorial
* SageMaker-Pytorth training Toolkit
    * https://github.com/aws/sagemaker-pytorch-training-toolkit/
* SageMaker-Pytorch Inference Toolkit
    * https://github.com/aws/sagemaker-pytorch-inference-toolkit
* SageMaker Inference Toolkit
    * https://docs.aws.amazon.com/sagemaker/latest/dg/amazon-sagemaker-toolkits.html
    * https://github.com/aws/sagemaker-inference-toolkit